In [1]:
cd ..

/home/alberto/Work/course_interpretability_deep_learning


# Multi-omics stratification on PDAC patients

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import optuna
import time
import dill
import shutil

from src import settings
from utils import RemoveFeaturesWithZeros, RemoveFeaturesWithNaN, FeatureSelectionNMF, RemoveCorrelatedFeatures, RemoveFeaturesLowMAE
from optimization import Optimization

## Load dataset

In [3]:
methylation_data = pd.read_csv(settings.methylation_data_path, sep=";", index_col=0, decimal=",")
methylation_data.columns = methylation_data.columns.str.replace(".", "-")
methylation_data = methylation_data.T
methylation_data = methylation_data.astype(np.float32)
print("methylation_data.shape", methylation_data.shape)
methylation_data.head()

methylation_data.shape (153, 301195)


,cg00000029,cg00000236,cg00000289,cg00000292,cg00000321,cg00000622,cg00000658,cg00000714,cg00000721,cg00000734,...,ch.9.2262725R,ch.9.2285199R,ch.9.2298007R,ch.9.2473665R,ch.9.357218F,ch.9.377428R,ch.9.691424R,ch.9.837340R,ch.9.898515R,ch.9.991104F
TCGA-2J-AAB6,0.157951,0.836226,0.710511,0.560780,0.239194,0.016433,0.864604,0.087681,0.938775,0.061008,...,0.103136,0.053757,0.032478,NaN,0.064965,0.049776,0.115268,0.095954,0.084203,NaN
TCGA-2J-AAB8,0.300754,0.782242,0.574296,0.670286,0.424310,0.014747,0.885958,0.112524,0.930765,0.037198,...,0.028180,0.054483,0.022736,NaN,0.060835,0.036434,0.160082,0.059216,0.065342,0.166304
TCGA-2J-AAB9,0.257807,0.846522,0.534748,0.688073,0.295597,0.014649,0.895039,0.167297,0.940112,0.058407,...,0.059313,0.063187,0.032581,NaN,0.055342,0.069086,0.128546,0.120015,0.074940,NaN
TCGA-2J-AABA,0.239086,0.789457,0.474723,0.705372,0.530321,0.016919,0.884874,0.129581,0.910885,0.062167,...,0.122677,0.056068,0.023190,0.109351,0.056015,0.053238,0.082979,0.057172,0.045781,0.121676
TCGA-2J-AABE,0.168622,0.841684,0.591205,0.623799,0.322576,0.014408,0.898202,0.125415,0.941153,0.059365,...,0.046699,0.049177,0.032707,NaN,0.075854,0.062602,0.122072,0.082753,0.071240,NaN


In [4]:
rnaseq_data = pd.read_csv(settings.rnaseq_data_path, sep=";", index_col=0, decimal=",")
rnaseq_data = rnaseq_data.T
rnaseq_data = rnaseq_data.astype(np.float32)
print("rnaseq_data.shape", rnaseq_data.shape)
rnaseq_data.head()

rnaseq_data.shape (147, 20501)


,A1BG,A1CF,A2BP1,A2LD1,A2ML1,A2M,A4GALT,A4GNT,AAA1,AAAS,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
TCGA-2J-AAB6,82.549698,8.187100,0.0000,163.122803,1815.789551,8517.444336,1121.052612,1.169600,1.1696,834.502930,...,14.619900,269.005798,1053.216431,0.5848,683.625671,11696.491211,869.005798,601.754395,26.315800,0.0000
TCGA-2J-AAB8,56.930698,33.842499,0.0000,185.814301,16.921301,14413.913086,392.949493,9.400700,0.9401,801.880127,...,35.722698,356.286713,829.142212,3.7603,680.611023,5829.377441,828.202087,609.165710,85.546402,0.0000
TCGA-2J-AAB9,105.787804,21.436199,1.0718,166.709503,642.015015,24311.779297,1125.401855,50.375099,0.0000,862.808105,...,57.877800,381.564789,936.763123,1.0718,646.302307,8094.319336,1083.601318,573.419128,30.010700,0.0000
TCGA-2J-AABA,99.345497,18.788200,0.0000,99.276703,873.649597,10302.006836,633.161072,6.262700,18.7882,623.767029,...,52.606899,293.721588,1511.820923,1.2525,945.670898,4829.810547,1364.646851,793.486816,31.313601,0.6263
TCGA-2J-AABE,79.401901,3.083100,0.0000,134.564499,74.610802,11076.861328,710.343811,35.147202,0.0000,702.327698,...,56.728802,431.632507,1069.215454,0.6166,564.205322,7464.775879,832.434082,468.629608,48.096199,0.0000


In [5]:
samples = methylation_data.index.intersection(rnaseq_data.index)
methylation_data = methylation_data.loc[samples]
rnaseq_data = rnaseq_data.loc[samples]
assert methylation_data.index.equals(rnaseq_data.index)
Xs= [rnaseq_data, methylation_data]
print("common samples:", len(samples))

common samples: 147


In [6]:
rnaseq_pipeline = make_pipeline(
    RemoveFeaturesWithZeros(threshold= 0.2, verbose= True),
    RemoveFeaturesLowMAE(percentage_to_keep= 0.5, verbose= True),
    RemoveCorrelatedFeatures(threshold = 0.85, verbose= True),
    FunctionTransformer(lambda x: np.log2(1 + x)),
    FeatureSelectionNMF(nmf = NMF(n_components= 500, max_iter=10000, random_state=settings.RANDOM_STATE), verbose= True),
    StandardScaler().set_output(transform= 'pandas'),
)
rnaseq_pipeline

Pipeline(steps=[('removefeatureswithzeros',
                 RemoveFeaturesWithZeros(verbose=True)),
                ('removefeatureslowmae',
                 RemoveFeaturesLowMAE(percentage_to_keep=0.5, verbose=True)),
                ('removecorrelatedfeatures',
                 RemoveCorrelatedFeatures(threshold=0.85, verbose=True)),
                ('functiontransformer',
                 FunctionTransformer(func=<function <lambda> at 0x7f2028d967a0>)),
                ('featureselectionnmf',
                 FeatureSelectionNMF(nmf=NMF(max_iter=10000, n_components=500,
                                             random_state=42),
                                     verbose=True)),
                ('standardscaler', StandardScaler())])

In [7]:
methylation_pipeline = make_pipeline(
    RemoveFeaturesWithNaN(threshold = 0.2, verbose= True),
    RemoveFeaturesLowMAE(percentage_to_keep= 0.1, verbose= True),
    # RemoveCorrelatedFeatures(threshold = 0.85, verbose= True),
    SimpleImputer(strategy= "mean").set_output(transform= 'pandas'),
    FeatureSelectionNMF(nmf = NMF(n_components= 250, max_iter=10000, random_state=settings.RANDOM_STATE), verbose= True),
    StandardScaler().set_output(transform= 'pandas'),
)
methylation_pipeline

Pipeline(steps=[('removefeatureswithnan', RemoveFeaturesWithNaN(verbose=True)),
                ('removefeatureslowmae',
                 RemoveFeaturesLowMAE(percentage_to_keep=0.1, verbose=True)),
                ('simpleimputer', SimpleImputer()),
                ('featureselectionnmf',
                 FeatureSelectionNMF(nmf=NMF(max_iter=10000, n_components=250,
                                             random_state=42),
                                     verbose=True)),
                ('standardscaler', StandardScaler())])

In [8]:
shutil.rmtree("lightning_logs/", ignore_errors= True)
shutil.rmtree("checkpoints/", ignore_errors= True)

In [9]:
study = optuna.create_study(direction="maximize")
new_study = True
if new_study:
    shutil.rmtree("tensorboard/", ignore_errors= True)
    date = time.strftime('%Y%m%d%H')
    study = optuna.create_study(direction="maximize")
    save_pipelines = True
    n_trials = 1
    for file in os.listdir(settings.results_path):
        os.remove(os.path.join(settings.results_path, file))
else:
    date = "2023062222"
    with open(os.path.join(settings.results_path, f'optimization_optuna_{date}.pkl'), 'rb') as file:
        study = dill.load(file)
    save_pipelines = False
    n_trials = 1000

[I 2023-06-23 00:41:23,519] A new study created in memory with name: no-name-f78ae8b3-eb46-41f6-a652-37f6e5dfe96b
[I 2023-06-23 00:41:24,010] A new study created in memory with name: no-name-628e68ee-186c-4310-8185-50d44d3eb137


In [ ]:
pipelines= [rnaseq_pipeline, methylation_pipeline]
func_objective = lambda trial: Optimization().objective(trial= trial, Xs= Xs, samples= samples, pipelines= pipelines, 
                                                        features_per_component_options= [2, 10, 2], num_layers_option= [1,2,1], num_units_option= [2,10,2],
                                                        n_epochs_option= [20,100,20], lambda_option = [0., 1., 0.25], n_clusters_option= [2,6,1],
                                                        random_state=settings.RANDOM_STATE, n_jobs= 4, save_pipelines= save_pipelines,  folder= settings.results_path)

keep_trying = True
while keep_trying:
    try:
        study = Optimization.optimize_optuna_and_save(study= study, n_trials = n_trials, date=date, show_progress_bar= True, folder= settings.results_path, func= func_objective)
    except FileNotFoundError:
        if new_study:
            keep_trying = False

  0%|          | 0/1 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasin

RemoveFeaturesWithZeros keeping 17112 features
RemoveFeaturesLowMAE keeping 8556 features
RemoveCorrelatedFeatures keeping 7075 features
FeatureSelectionNMF keeping 5000 features
RemoveFeaturesWithNaN keeping 300520 features
RemoveFeaturesLowMAE keeping 30052 features
FeatureSelectionNMF keeping 2500 features


Finding best initial lr:  98%|█████████▊| 98/100 [00:01<00:00, 77.86it/s]
LR finder stopped early after 98 steps due to diverging loss.
Learning rate set to 0.0013182567385564075
Restoring states from the checkpoint path at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_5ea43220-5b59-44eb-8403-eec5b6c4e617.ckpt
Restored all states from the checkpoint at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_5ea43220-5b59-44eb-8403-eec5b6c4e617.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 wh

RemoveFeaturesWithZeros keeping 17096 features
RemoveFeaturesLowMAE keeping 8548 features
RemoveCorrelatedFeatures keeping 6985 features
FeatureSelectionNMF keeping 5000 features
RemoveFeaturesWithNaN keeping 300793 features
RemoveFeaturesLowMAE keeping 30079 features
FeatureSelectionNMF keeping 2500 features


...

Finding best initial lr: 100%|██████████| 100/100 [00:00<00:00, 141.34it/s]
Learning rate set to 0.005754399373371567
Restoring states from the checkpoint path at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_82e9bbe9-9708-40ef-9c95-0dc2bd36f566.ckpt
Restored all states from the checkpoint at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_82e9bbe9-9708-40ef-9c95-0dc2bd36f566.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader

RemoveFeaturesWithZeros keeping 17141 features
RemoveFeaturesLowMAE keeping 8570 features
RemoveCorrelatedFeatures keeping 6886 features
FeatureSelectionNMF keeping 1000 features
RemoveFeaturesWithNaN keeping 300874 features
RemoveFeaturesLowMAE keeping 30087 features
FeatureSelectionNMF keeping 500 features


Finding best initial lr: 100%|██████████| 100/100 [00:00<00:00, 140.83it/s]
Learning rate set to 0.008317637711026709
Restoring states from the checkpoint path at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_19984af7-5a7b-4557-8110-0502f7a112ab.ckpt
Restored all states from the checkpoint at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_19984af7-5a7b-4557-8110-0502f7a112ab.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader

RemoveFeaturesWithZeros keeping 17132 features
RemoveFeaturesLowMAE keeping 8566 features
RemoveCorrelatedFeatures keeping 6944 features
FeatureSelectionNMF keeping 1000 features
RemoveFeaturesWithNaN keeping 300756 features
RemoveFeaturesLowMAE keeping 30075 features
FeatureSelectionNMF keeping 500 features


Finding best initial lr: 100%|██████████| 100/100 [00:00<00:00, 264.11it/s]
Learning rate set to 0.005754399373371567
Restoring states from the checkpoint path at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_058c988d-a5a3-42a6-8265-7080ac3ba7d9.ckpt
Restored all states from the checkpoint at /home/alberto/Work/course_interpretability_deep_learning/.lr_find_058c988d-a5a3-42a6-8265-7080ac3ba7d9.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/alberto/anaconda3/envs/course_interpretability_dl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader

In [ ]:
study.best_params

In [8]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

NameError: name 'study' is not defined

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_slice(study, params= ["num_layers", "divisor_view0_layer1", "divisor_view1_layer1"])
fig.show()

In [ ]:
params = [i for i in list(study.best_params.keys()) if i not in ["num_layers", "divisor_view0_layer1", "divisor_view1_layer1"]]
fig = optuna.visualization.plot_param_importances(study, params= params)
fig.show()

In [ ]:
fig = optuna.visualization.plot_slice(study, params= params)
fig.show()

In [9]:
pipelines = []
transformed_Xs = []
for idx_pipeline, X in enumerate(Xs):
    pipeline_name = f"pipeline{idx_pipeline}.pkl"
    with open(os.path.join(settings.results_path, pipeline_name), 'rb') as f:
        pipeline = dill.load(f)
    transformed_X = pipeline.transform(X)
    transformed_Xs.append(transformed_X)
    pipelines.append(pipeline)
transformed_X = pd.concat(transformed_Xs, axis = 1)
print("transformed_X.shape", transformed_X.shape)
transformed_X.head()

transformed_Xs.shape (147, 1500)


,A4GNT,AADAC,AATK,ABCA7,ABCD3,ABO,ABP1,ACHE,ACSL5,ADAM9,...,cg27101125,cg27156542,cg27205687,cg27285720,cg27286337,cg27341866,cg27351978,cg27363327,cg27518014,cg27520776
TCGA-2J-AAB6,-1.456777,-1.937763,-0.515733,1.113723,-1.569327,-1.769025,-1.653193,-2.027797,-0.372280,0.483969,...,-0.833185,-3.380069,1.868786,-0.502744,-1.075593e+00,-1.405487,0.197165,1.658391,0.033656,0.625561
TCGA-2J-AAB8,-0.597544,0.443216,0.302752,1.176438,0.011658,0.498676,0.167327,-0.794278,0.947392,-0.761113,...,-0.678101,0.834531,0.138608,0.045512,-1.403588e+00,-1.005612,0.733370,-0.051521,0.415313,-0.535517
TCGA-2J-AAB9,0.278107,0.097128,0.481229,-0.185232,-0.594198,-0.906301,-0.252470,-0.075807,-1.076427,-0.041190,...,-0.751646,0.671349,-1.328237,1.988451,9.113319e-01,-0.522277,0.036797,-0.782170,0.238930,-0.732899
TCGA-2J-AABA,-0.794420,-0.498316,0.639924,0.508077,1.535674,0.125791,0.854734,-0.377861,0.466614,-0.177544,...,0.191473,0.848614,-1.362668,-0.070065,7.377306e-01,-0.502348,0.158920,-0.903573,0.053332,-0.670505
TCGA-2J-AABE,0.085380,0.292508,0.976312,1.017757,-1.496673,1.744971,0.158441,0.609354,1.641325,-0.938477,...,1.278028,0.771772,-1.282736,0.234885,-5.335914e-07,0.429864,0.509858,-0.859494,0.471717,0.046852


In [ ]:
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from umap import UMAP

In [94]:
clusters = np.random.randint(0, 2, len(transformed_Xs))

In [105]:
method = PCA(n_components = 2, random_state = settings.RANDOM_STATE)
plot_data = method.fit_transform(transformed_X)
fig = px.scatter(x = plot_data[:, 0], y = plot_data[:, 1], color = clusters)
fig.show()

In [106]:
method = TSNE(n_components = 2, random_state = settings.RANDOM_STATE)
plot_data = method.fit_transform(transformed_X)
fig = px.scatter(x = plot_data[:, 0], y = plot_data[:, 1], color = clusters)
fig.show()

In [107]:
method = UMAP(n_components = 2, random_state = settings.RANDOM_STATE)
plot_data = method.fit_transform(transformed_X)
fig = px.scatter(x = plot_data[:, 0], y = plot_data[:, 1], color = clusters)
fig.show()

In [115]:
method = PCA(n_components = 3, random_state = settings.RANDOM_STATE)
plot_data = method.fit_transform(transformed_X)
fig = px.scatter_3d(x = plot_data[:, 0], y = plot_data[:, 1], z = plot_data[:, 2], color = clusters)
fig.show()

In [116]:
method = TSNE(n_components = 3, random_state = settings.RANDOM_STATE)
plot_data = method.fit_transform(transformed_X)
fig = px.scatter_3d(x = plot_data[:, 0], y = plot_data[:, 1], z = plot_data[:, 2], color = clusters)
fig.show()

In [117]:
method = UMAP(n_components = 3, random_state = settings.RANDOM_STATE)
plot_data = method.fit_transform(transformed_X)
fig = px.scatter_3d(x = plot_data[:, 0], y = plot_data[:, 1], z = plot_data[:, 2], color = clusters)
fig.show()